In [ ]:
#training musica classica

In [ ]:
""" This file permits to create the Data set from midi files and train_setup the neural network """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model


def main():
    # use this to transform midi files into encoded list of notes/chords
    #notes = get_notes()

    #  use this to load encoded notes/chords list from a pickle file
    notes = []
    # this path must be changed!
    with (open("/content/drive/MyDrive/Università/Anno4/Natural Interaction/data/jazz", "rb")) as openfile:
        notes = pickle.load(openfile)

    # this is the length of unique notes/chords (number of elements in out Vocabulary Domain)
    n_elems = len(set(notes))

    network_input, network_output = sequences_generator(notes, n_elems)

    model = build_network(network_input, n_elems)

    train_setup(model, network_input, network_output)


def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    # this path must be changed!
    for file in glob.glob("/content/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    # this path  must be changed!
    with open('/content/drive/MyDrive/Università/Anno4/Natural Interaction/data/jazz', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def sequences_generator(notes, n_elems):
    """ Prepare the sequences used by the neural network"""
    sequence_length = 100

    # get all unique notes/chords
    elem_collection = sorted(set(item for item in notes))

    # create a dictionary to map elements to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(elem_collection))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_elems)
    # one-hot encoding of the outputs
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def build_network(network_input, n_elems):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_elems))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    #plot_model(model, to_file='lstm.png',
    #           show_shapes=True, show_layer_names=True)
    model.summary()
    return model


def train_setup(model, network_input, network_output):
    """ train the neural network with SGD """
    # this paths must be changed!
    dir = "/content/drive/MyDrive/Università/Anno4/Natural Interaction/ckp/"
    filepath = dir+"checkpoint-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=1,
              batch_size=128, callbacks=callbacks_list)


if __name__ == '__main__':
    main()


In [ ]:
### generazione musica classica

In [ ]:
"""This file generates notes/chords and save them as midi file"""
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation


def main():
    # this path must be changed!
    # first I load the notes-pickle file that will be used to choose a random sequence for starting generation of new elements
    with open('/content/drive/MyDrive/Università/Anno4/Natural Interaction/data/classic', 'rb') as filepath:
        notes = pickle.load(filepath)

    # get all unique notes/chords
    elem_collection = sorted(set(item for item in notes))
    # this is the length of unique notes/chords (number of elements in out Vocabulary Domain)
    n_elems = len(set(notes))

    network_input, normalized_input = sequences_generator(
        notes, elem_collection, n_elems)
    model = build_network(normalized_input, n_elems)
    for i in range(5):
      s = "out{}.mid".format(i)
      prediction_output = generate_notes(
          model, network_input, elem_collection, n_elems)
      create_midi(prediction_output,s)


def sequences_generator(notes, elem_collection, n_elems):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number)
                       for number, note in enumerate(elem_collection))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_elems)

    return (network_input, normalized_input)


def build_network(network_input, n_elems):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_elems))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    dir = "/content/drive/MyDrive/Università/Anno4/Natural Interaction/ckp/"
    model.load_weights(dir+'classic.hdf5')

    return model


def generate_notes(model, network_input, elem_collection, n_elems):
    """ Generate notes from the neural network based on an initial sequence of notes """
    # first we take a random sequence from the input as a starting point for the generation
    numpy.random.seed(int(time.time()))
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note)
                       for number, note in enumerate(elem_collection))

    pattern = network_input[start]
    prediction_output = []

    # generate 'n' notes
    for _ in range(150):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_elems)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


def create_midi(prediction_output, name):
    """ Convert the predictions output to notes and then create a midi file """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for elem in prediction_output:
        # elem is a chord
        if ('.' in elem) or elem.isdigit():
            notes_in_chord = elem.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # elem is a note
        else:
            new_note = note.Note(elem)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        # offset refers to where the note is located in the piece.
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=name)


if __name__ == '__main__':
    main()


In [ ]:
#training jazz

In [ ]:
""" This file permits to create the Data set from midi files and train_setup the neural network """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model


def main():
    # use this to transform midi files into encoded list of notes/chords
    #notes = get_notes()

    #  use this to load encoded notes/chords list from a pickle file
    notes = []
    # this path must be changed!
    with (open("data/jazz", "rb")) as openfile:
        notes = pickle.load(openfile)
    notes = notes[:int(len(notes)/5)]

    # this is the length of unique notes/chords (number of elements in out Vocabulary Domain)
    n_elems = len(set(notes))

    network_input, network_output = sequences_generator(notes, n_elems)

    model = build_network(network_input, n_elems)

    train_setup(model, network_input, network_output)


def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    # this path must be changed!
    for file in glob.glob("midi/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    # this path  must be changed!
    with open('data/jazz', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def sequences_generator(notes, n_elems):
    """ Prepare the sequences used by the neural network"""
    sequence_length = 80

    # get all unique notes/chords
    elem_collection = sorted(set(item for item in notes))

    # create a dictionary to map elements to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(elem_collection))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_elems)
    # one-hot encoding of the outputs
    network_output = to_categorical(network_output)

    return (network_input, network_output)


def build_network(network_input, n_elems):
    model = Sequential()
    model.add(LSTM(
        704,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(704, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(704))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(640))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_elems))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    # plot_model(model, to_file='lstm.png',
    #           show_shapes=True, show_layer_names=True)
    model.summary()
    return model


def train_setup(model, network_input, network_output):
    """ train the neural network with SGD """
    # this paths must be changed!
    dir = "ckp/"
    filepath = dir+"checkpoint-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=1000,
              batch_size=128, callbacks=callbacks_list)


if __name__ == '__main__':
    main()


In [ ]:
#generazione jazz

In [ ]:
"""This file generates notes/chords and save them as midi file"""
import pickle
import numpy
import time
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation


def main():
    # this path must be changed!
    # first I load the notes-pickle file that will be used to choose a random sequence for starting generation of new elements
    with open('/content/drive/MyDrive/Università/Anno4/Natural Interaction/data/jazz', 'rb') as filepath:
        notes = pickle.load(filepath)
    notes = notes[:int(len(notes)/5)]

    # get all unique notes/chords
    elem_collection = sorted(set(item for item in notes))
    # this is the length of unique notes/chords (number of elements in out Vocabulary Domain)
    n_elems = len(set(notes))

    network_input, normalized_input = sequences_generator(
        notes, elem_collection, n_elems)
    model = build_network(normalized_input, n_elems)
    for i in range(5):
      s = "out{}.mid".format(i)
      prediction_output = generate_notes(
          model, network_input, elem_collection, n_elems)
      create_midi(prediction_output,s)


def sequences_generator(notes, elem_collection, n_elems):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number)
                       for number, note in enumerate(elem_collection))

    sequence_length = 80
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_elems)

    return (network_input, normalized_input)


def build_network(network_input, n_elems):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        704,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(704, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(704))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(640))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_elems))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    dir = "/content/drive/MyDrive/Università/Anno4/Natural Interaction/ckp/"
    model.load_weights(dir+'jazz.hdf5')

    return model


def generate_notes(model, network_input, elem_collection, n_elems):
    """ Generate notes from the neural network based on an initial sequence of notes """
    # first we take a random sequence from the input as a starting point for the generation
    numpy.random.seed(int(time.time()))
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note)
                       for number, note in enumerate(elem_collection))

    pattern = network_input[start]
    prediction_output = []

    # generate 'n' notes
    for _ in range(150):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_elems)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


def create_midi(prediction_output, name):
    """ Convert the predictions output to notes and then create a midi file """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for elem in prediction_output:
        # elem is a chord
        if ('.' in elem) or elem.isdigit():
            notes_in_chord = elem.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # elem is a note
        else:
            new_note = note.Note(elem)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        # offset refers to where the note is located in the piece.
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=name)


if __name__ == '__main__':
    main()


In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [17]:
!fluidsynth -ni font.sf2 out0.mid -F out0.wav -r 44100
!fluidsynth -ni font.sf2 out1.mid -F out1.wav -r 44100
!fluidsynth -ni font.sf2 out2.mid -F out2.wav -r 44100
!fluidsynth -ni font.sf2 out3.mid -F out3.wav -r 44100
!fluidsynth -ni font.sf2 out4.mid -F out4.wav -r 44100

In [ ]:
from IPython.display import Audio 
Audio('out0.wav')

In [ ]:
from IPython.display import Audio 
Audio('out1.wav')

In [ ]:
from IPython.display import Audio 
Audio('out2.wav')

In [ ]:
from IPython.display import Audio 
Audio('out3.wav')

In [ ]:
from IPython.display import Audio 
Audio('out4.wav')